In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
findspark.init('/usr/local/opt/apache-spark')

In [11]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Spark in Jupyter") \
    .getOrCreate()

KeyboardInterrupt: 

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
df = spark.read.csv(url, header=True, inferSchema=True)
df.show()

NameError: name 'spark' is not defined

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

NameError: name 'df' is not defined

In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
four_bedroom_df = df.filter(col("bedrooms") == 4)
four_bedroom_df = four_bedroom_df.withColumn("year", col("date").substr(1, 4).cast("int"))
# Group by year and calculate the average price
average_price_per_year_df = four_bedroom_df.groupBy("year").agg(
    round(col("price").cast("double").avg(), 2).alias("average_price")
)

# Show the result
average_price_per_year_df.show()


NameError: name 'df' is not defined

In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
three_bedroom_df = df.filter((col("bedrooms") == 3) & (col("bathrooms") == 3))

# Group by the year built and calculate the average price
average_price_per_year_built_df = three_bedroom_df.groupBy("year_built").agg(
    round(col("price").cast("double").avg(), 2).alias("average_price")
)
# result
average_price_per_year_built_df.show()

In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

homes_3bd_3ba_2floors_2000sqft = df.filter(
    (col("bedrooms") == 3) &
    (col("bathrooms") == 3) &
    (col("floors") == 2) &
    (col("square_feet") >= 2000)
)

# Group by the year the home was built and calculate the average price
average_price_per_year_built_df = homes_3bd_3ba_2floors_2000sqft.groupBy("year_built").agg(
    round(col("price").cast("double").avg(), 2).alias("average_price")
)

# result
average_price_per_year_built_df.show()

In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

# Start time
start_time = time.time()

# Filter the DataFrame for homes with average price >= $350,000
homes_above_350k_df = df.filter(col("price") >= 350000)

# Group by the 'view' rating and calculate the average price
average_price_per_view_df = homes_above_350k_df.groupBy("view").agg(
    round(col("price").cast("double").avg(), 2).alias("average_price")
).orderBy(col("view").desc())

# Show the result
average_price_per_view_df.show()

# End time
end_time = time.time()

# Print runtime
print("--- %s seconds ---" % (end_time - start_time))

--- 7.796287536621094e-05 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [ ]:
# 8. Check if the table is cached.
print("Cached Tables:")
print(spark.catalog.listTables())

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

# Start time for cached query
start_time_cached = time.time()

# Run the query on cached data
average_price_per_view_cached_df = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""")

# Show the result
average_price_per_view_cached_df.show()

# End time for cached query
end_time_cached = time.time()

# Print runtime for cached query
print("--- Cached Query Runtime: %s seconds ---" % (end_time_cached - start_time_cached))

# Start time for uncached query
start_time_uncached = time.time()

# Run the query on uncached data
average_price_per_view_uncached_df = df.filter(col("price") >= 350000).groupBy("view").agg(
    round(col("price").cast("double").avg(), 2).alias("average_price")
).orderBy(col("view").desc())

# Show the result
average_price_per_view_uncached_df.show()

# End time for uncached query
end_time_uncached = time.time()

# Print runtime for uncached query
print("--- Uncached Query Runtime: %s seconds ---" % (end_time_uncached - start_time_uncached))

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
output_path = "path"

# Write the DataFrame to Parquet format, partitioning by 'date_built'
df.write.partitionBy("date_built").parquet(output_path)

print("Data written to Parquet format and partitioned by 'date_built'")

In [ ]:
# 11. Read the formatted parquet data.
input_path = "path"

# Read the partitioned Parquet data into a DataFrame
df_parquet = spark.read.parquet(input_path)

# Show the first few rows of the DataFrame to verify
df_parquet.show()

# Optionally, print the schema of the DataFrame
df_parquet.printSchema()

In [ ]:
# 12. Create a temporary table for the parquet data.
input_path = "path"
df_parquet = spark.read.parquet(input_path)

# Create a temporary view of the DataFrame
df_parquet.createOrReplaceTempView("home_sales_parquet")

# Verify that the temporary view is created by listing the tables
print("Temporary Views:")
print(spark.catalog.listTables())

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

input_path = "path_to_output_directory/home_sales_partitioned"
df_parquet = spark.read.parquet(input_path)
df_parquet.createOrReplaceTempView("home_sales_parquet")

# Start time for query on Parquet data
start_time_parquet = time.time()

# Run the query on the Parquet data
average_price_per_view_parquet_df = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales_parquet
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""")

# Show the result
average_price_per_view_parquet_df.show()

# End time for query on Parquet data
end_time_parquet = time.time()

# Print runtime for query on Parquet data
print("--- Parquet Query Runtime: %s seconds ---" % (end_time_parquet - start_time_parquet))

cached_query_runtime = 5.0  # Replace this with the actual cached query runtime

# Print runtime comparison
print("--- Parquet Query Runtime: %s seconds ---" % (end_time_parquet - start_time_parquet))
print("--- Cached Query Runtime: %s seconds ---" % cached_query_runtime)

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

# Verify that the table is uncached by listing cached tables
print("Cached Tables After Uncaching:")
print(spark.catalog.listTables())

In [ ]:
# 15. Check if the home_sales is no longer cached
print("Cached Tables After Uncaching:")
tables_after_uncaching = spark.catalog.listTables()
for table in tables_after_uncaching:
    print(f"Table Name: {table.name}, Is Temporary: {table.isTemporary}, Is Cached: {table.isCached}")